# 准备数据

In [22]:
# 加载模块
from datetime import datetime

from tqdm import tqdm
from xtquant import xtdata

from vnpy.trader.database import DB_TZ
from vnpy.trader.datafeed import get_datafeed
from vnpy.trader.constant import Exchange, Interval
from vnpy.trader.object import HistoryRequest

from vnpy.alpha import AlphaLab, logger

In [23]:
# 设置下载参数
task_name = "csi300"
index_symbol = "000300.SSE"
xt_index_symbol = "000300.SH"

start_date = "20070101"
end_date = "20231231"

intervals = [
    Interval.DAILY,
]

In [24]:
# 创建投研实验室
lab = AlphaLab(f"./lab/{task_name}")    # 指定数据文件夹

In [25]:
# 初始化数据服务（这里配置使用的迅投研）
datafeed = get_datafeed()
datafeed.init()

True

In [27]:
# 下载历史成分股信息
xtdata.download_sector_data()

# xtdata.download_history_data("", "stocklistchange", "", "")

In [32]:
# 查询交易日历（替代方案：定期采样获取成分股）
# 注意：get_trading_calendar()在当前客户端版本不支持，改用定期采样方式
from datetime import timedelta

# 生成每月第一天的日期列表（成分股通常每月调整一次，减少API调用）
days = []
start_dt = datetime.strptime(start_date, "%Y%m%d")
end_dt = datetime.strptime(end_date, "%Y%m%d")
current = start_dt

while current <= end_dt:
    days.append(current.strftime("%Y%m%d"))
    # 移动到下个月第一天
    if current.month == 12:
        current = current.replace(year=current.year + 1, month=1, day=1)
    else:
        current = current.replace(month=current.month + 1, day=1)

# 轮询获取指数成分股
index_components = {}
end_datetime = datetime.strptime(end_date, "%Y%m%d")

for i in days:
    dt = datetime.strptime(i, "%Y%m%d")
    if dt > end_datetime:
        continue

    try:
        # 使用xtdata.get_stock_list_in_sector获取成分股
        xt_symbols = xtdata.get_stock_list_in_sector(xt_index_symbol, i)
        
        if xt_symbols:  # 如果成功获取到数据
            vt_symbols: list = []
            for xt_symbol in xt_symbols:
                vt_symbol = xt_symbol.replace("SH", "SSE").replace("SZ", "SZSE")
                vt_symbols.append(vt_symbol)
            
            index_components[dt.strftime("%Y-%m-%d")] = vt_symbols
            logger.info(f"成功获取{i}的成分股，共{len(vt_symbols)}只")
    except Exception as e:
        # 如果某天获取失败，跳过（可能是非交易日）
        logger.warning(f"获取{i}的成分股失败: {e}")
        continue

# 保存到数据中心
lab.save_component_data(index_symbol, index_components)
logger.info(f"成分股数据已保存，共{len(index_components)}个交易日")

2025-12-31 22:38:11 成功获取20070101的成分股，共300只
2025-12-31 22:38:11 成功获取20070201的成分股，共300只
2025-12-31 22:38:11 成功获取20070301的成分股，共300只
2025-12-31 22:38:11 成功获取20070401的成分股，共300只
2025-12-31 22:38:11 成功获取20070501的成分股，共300只
2025-12-31 22:38:11 成功获取20070601的成分股，共300只
2025-12-31 22:38:11 成功获取20070701的成分股，共300只
2025-12-31 22:38:11 成功获取20070801的成分股，共300只
2025-12-31 22:38:11 成功获取20070901的成分股，共300只
2025-12-31 22:38:11 成功获取20071001的成分股，共300只
2025-12-31 22:38:11 成功获取20071101的成分股，共300只
2025-12-31 22:38:11 成功获取20071201的成分股，共300只
2025-12-31 22:38:11 成功获取20080101的成分股，共300只
2025-12-31 22:38:11 成功获取20080201的成分股，共300只
2025-12-31 22:38:11 成功获取20080301的成分股，共300只
2025-12-31 22:38:11 成功获取20080401的成分股，共300只
2025-12-31 22:38:11 成功获取20080501的成分股，共300只
2025-12-31 22:38:11 成功获取20080601的成分股，共300只
2025-12-31 22:38:11 成功获取20080701的成分股，共300只
2025-12-31 22:38:11 成功获取20080801的成分股，共300只
2025-12-31 22:38:11 成功获取20080901的成分股，共300只
2025-12-31 22:38:11 成功获取20081001的成分股，共300只
2025-12-31 22:38:11 成功获取20081101的成分股，共300只
2025-12-31 

In [33]:
# 加载指数成分股代码
component_symbols = lab.load_component_symbols(index_symbol, start_date, end_date)

In [39]:
# 转换时间格式
start = datetime.strptime(start_date, "%Y%m%d")
start = start.replace(tzinfo=DB_TZ)

end = datetime.strptime(end_date, "%Y%m%d")
end = end.replace(tzinfo=DB_TZ)

# 除了成分股，还要下载指数数据
task_symbols = component_symbols + [index_symbol]

# 统计成功和失败数量
success_count = 0
fail_count = 0
failed_symbols = []

# 根据interval转换为xt的period格式
period_map = {
    Interval.DAILY: "1d",
    Interval.HOUR: "1h",
    Interval.MINUTE: "1m",
}

# 遍历下载数据
for vt_symbol in tqdm(task_symbols):
    symbol, exchange_str = vt_symbol.split(".")
    
    # 转换vt_symbol格式为xt格式（用于xtdata下载）
    # vt_symbol格式: 600000.SSE -> xt格式: 600000.SH
    xt_symbol = symbol + (".SH" if exchange_str == "SSE" else ".SZ")

    for interval in intervals:
        period = period_map.get(interval, "1d")
        bars = None
        
        # 方法1：先通过xtdata下载数据到本地（迅投研需要先下载到本地）
        try:
            xtdata.download_history_data(xt_symbol, period=period, incrementally=True)
        except Exception as e:
            logger.warning(f"下载{vt_symbol}的本地数据失败: {e}")
        
        # 方法2：尝试通过datafeed查询数据
        try:
            req = HistoryRequest(symbol, Exchange(exchange_str), start, end, interval)
            bars = datafeed.query_bar_history(req)
        except Exception as e:
            logger.debug(f"通过datafeed查询{vt_symbol}失败: {e}")
        
        # 方法3：如果datafeed返回空，直接使用xtdata获取数据
        if not bars:
            try:
                # 直接使用xtdata获取数据
                import pandas as pd
                from vnpy.trader.object import BarData
                
                # 获取数据（返回DataFrame格式）
                df = xtdata.get_market_data_ex([], [xt_symbol], period=period, count=-1)
                
                if df is not None and not df.empty and xt_symbol in df:
                    symbol_data = df[xt_symbol]
                    if not symbol_data.empty:
                        # 转换为BarData列表
                        bars = []
                        for idx, row in symbol_data.iterrows():
                            # 转换时间格式
                            if isinstance(idx, pd.Timestamp):
                                bar_dt = idx.to_pydatetime()
                            else:
                                bar_dt = datetime.strptime(str(idx), "%Y%m%d") if len(str(idx)) == 8 else datetime.fromtimestamp(idx)
                            
                            bar_dt = bar_dt.replace(tzinfo=DB_TZ)
                            
                            # 创建BarData对象
                            bar = BarData(
                                symbol=symbol,
                                exchange=Exchange(exchange_str),
                                datetime=bar_dt,
                                interval=interval,
                                open_price=float(row.get('open', 0)),
                                high_price=float(row.get('high', 0)),
                                low_price=float(row.get('low', 0)),
                                close_price=float(row.get('close', 0)),
                                volume=float(row.get('volume', 0)),
                                turnover=float(row.get('amount', 0)),
                                open_interest=float(row.get('openInterest', 0)),
                                gateway_name="xt"
                            )
                            bars.append(bar)
                        
                        # 过滤时间范围
                        bars = [b for b in bars if start <= b.datetime <= end]
            except Exception as e:
                logger.debug(f"通过xtdata直接获取{vt_symbol}失败: {e}")
        
        # 保存数据
        if bars:
            lab.save_bar_data(bars)
            success_count += 1
        else:
            fail_count += 1
            failed_symbols.append(vt_symbol)
            logger.error(f"下载{vt_symbol}的{interval}数据失败：所有方法都返回空")

# 输出统计信息
logger.info(f"数据下载完成：成功{success_count}个，失败{fail_count}个")
if failed_symbols:
    logger.warning(f"失败的股票代码（前10个）：{failed_symbols[:10]}")

  0%|          | 0/301 [00:00<?, ?it/s]

2025-12-31 22:50:39 通过xtdata直接获取000776.SZSE失败: 'dict' object has no attribute 'empty'
2025-12-31 22:50:39 下载000776.SZSE的Interval.DAILY数据失败：所有方法都返回空
2025-12-31 22:50:39 通过xtdata直接获取001979.SZSE失败: 'dict' object has no attribute 'empty'
2025-12-31 22:50:39 下载001979.SZSE的Interval.DAILY数据失败：所有方法都返回空


  1%|          | 2/301 [00:00<00:24, 12.44it/s]

2025-12-31 22:50:39 通过xtdata直接获取601211.SSE失败: 'dict' object has no attribute 'empty'
2025-12-31 22:50:39 下载601211.SSE的Interval.DAILY数据失败：所有方法都返回空
2025-12-31 22:50:39 通过xtdata直接获取600018.SSE失败: 'dict' object has no attribute 'empty'
2025-12-31 22:50:39 下载600018.SSE的Interval.DAILY数据失败：所有方法都返回空


  1%|▏         | 4/301 [00:00<00:23, 12.54it/s]

...

2025-12-31 22:50:48 通过xtdata直接获取601229.SSE失败: 'dict' object has no attribute 'empty'
2025-12-31 22:50:48 下载601229.SSE的Interval.DAILY数据失败：所有方法都返回空
2025-12-31 22:50:48 通过xtdata直接获取688008.SSE失败: 'dict' object has no attribute 'empty'
2025-12-31 22:50:48 下载688008.SSE的Interval.DAILY数据失败：所有方法都返回空


 37%|███▋      | 112/301 [00:09<00:15, 12.55it/s]

2025-12-31 22:50:48 通过xtdata直接获取600886.SSE失败: 'dict' object has no attribute 'empty'
2025-12-31 22:50:48 下载600886.SSE的Interval.DAILY数据失败：所有方法都返回空


 38%|███▊      | 113/301 [00:09<00:15, 12.34it/s]


KeyboardInterrupt: 

In [40]:
# 添加回测参数配置
for vt_symbol in component_symbols:
    lab.add_contract_setting(
        vt_symbol,
        long_rate=5/10000,
        short_rate=10/10000,
        size=1,
        pricetick=0.0001,
    )